In [ ]:
import networkx as nx
import ipaddress

from string import Template

import yaml

In [ ]:
class dotdict(dict):
    """
    Access dictionary member with the dot operator
    
    dot.notation access to dictionary attributes
    from https://stackoverflow.com/a/23689767
    """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [ ]:
def min_max_range(_range):
    """
    Return the minimum and maximum value of a range
    """
    return tuple(map(int, _range.split('-')))

In [ ]:
class Topo:
    def __init__(self):
        self.reset()

    def reset(self):
        with open('template.yaml') as file:
            self.template = dotdict(yaml.load(file, Loader=yaml.FullLoader))
        self.G = nx.MultiGraph()

        self.counters = dotdict({'spines': 0, 'leafs': 0, 'management_addresses': 0, 'overlay_addresses': 0})

    def addSpine(self):
        next_spine = self.counters.spines + 1
        asn = self.template['spine']['ASN']
        name = Template(self.template['spine']['name']).substitute(id=next_spine)
        router_id = str(ipaddress.ip_network(self.template['spine']['router-id'])[next_spine])

        params = {'type': 'spine', 'name': name, 'ASN': asn, 'router-id': router_id}

        self.counters.spines =  self.counters.spines + 1

        return self.addNode(name, params)

    def _rack_asn(self, rack):
        (a,b) = min_max_range(self.template['rack']['ASN'])
        base_asn = a + rack
        if base_asn > b:
            raise Exception('You exceeded the maximum possible number of racks (max is: {})'.format(b-a+1))
        return base_asn

    def addLeaf(self, rack):
        next_leaf = self.counters.leafs + 1
        asn = self._rack_asn(rack)
        name = Template(self.template['leaf']['name']).substitute(id=next_leaf)
        router_id = str(ipaddress.ip_network(self.template['leaf']['router-id'])[next_leaf])

        params = {'type': 'leaf', 'name': name, 'ASN': asn, 'router-id': router_id, 'rack': rack}

        self.counters.leafs =  self.counters.leafs + 1

        return self.addNode(name, params)

    def getLeafs(self):
        return self.getNodesByType('leaf')
    def getSpines(self):
        return self.getNodesByType('spine')

    def getNodesByType(self, _type):
        return filter (lambda node:  node[1]['type']  == _type, self.G.nodes(data=True))

    def connect(self, a, aport, b, bport, attr={}):
        interfaces = {'ports': {a: aport, b: bport}}
        self.G.add_edge(a, b, **{**interfaces, **attr})

    def addNode(self, name, params):
        management_ip = self.generateManagementAddress()
        management_port = self.template['management']['interface'] 
        mgmt = {'management-ip': management_ip, 'management-port': management_port}

        _params = { **mgmt, **params}

        self.G.add_node(name, **_params)

        return (params['name'], params)

    def _generateAddress(self, discriminator, ip_range, netmask=None):
        index = self.counters[discriminator]
        low, high = map(ipaddress.ip_address, ip_range.split('-'))

        ip = low + index
        if ip > high:
            raise Exception('All management addresses are taken')

        if netmask is not None:
            prefixlen = ipaddress.IPv4Network('0.0.0.0/{}'.format(netmask)).prefixlen
            return '{}/{}'.format(ip, prefixlen)
        else:
            return '{}'.format(ip)


    def generateManagementAddress(self):
        address = self._generateAddress(discriminator= 'management_addresses', ip_range=self.template['management']['ip-range'], netmask=self.template['management']['netmask'])
        self.counters.management_addresses = self.counters.management_addresses + 1
        return address
    def generateOverlaySourceAddress(self):
        address = self._generateAddress(discriminator='overlay_addresses', ip_range=self.template['overlay']['source-ip'])
        self.counters.overlay_addresses = self.counters.overlay_addresses + 1
        return address

    def connectVlan(self, node, vid, network, interfaces=[]):
        overlay_source_address = self.G.nodes[node].get('overlay-source-address', self.generateOverlaySourceAddress())

        vlans = self.G.nodes[node].get('vlans', {})
        vnid = self.template['overlay']['base-vnid'] + vid

        net = ipaddress.ip_network(network)

        vlans[vid] = {
                        'vid': vid,
                        'vnid': vnid,
                        'network': network,
                        'interfaces': interfaces,
                        'anycast-ip-address': '{}/{}'.format(net.broadcast_address - 1, net.prefixlen),
                     }

        self.G.nodes[node]['overlay-source-address'] = overlay_source_address
        self.G.nodes[node]['vlans'] = vlans
        self.G.nodes[node]['anycast-mac-address'] = self.template['overlay']['anycast-mac-address']


In [ ]:
topo = Topo()
spine1, _ = topo.addSpine()
spine2, _ = topo.addSpine()
leaf1, _ = topo.addLeaf(0)
leaf2, _ = topo.addLeaf(1)
leaf3, _ = topo.addLeaf(2)

topo.connect(spine1, 'Eth1/2', leaf1, 'Eth1/2')
topo.connect(spine1, 'Eth1/3', leaf2, 'Eth1/2')
topo.connect(spine1, 'Eth1/4', leaf3, 'Eth1/2')

topo.connectVlan(node=leaf1, network= '192.168.40.0/24', vid=40, interfaces=['Eth1/7'])
topo.connectVlan(node=leaf2, network= '192.168.40.0/24', vid=40, interfaces=['Eth1/7'])
topo.connectVlan(node=leaf3, network= '192.168.40.0/24', vid=40, interfaces=['Eth1/7'])

In [ ]:
def _cmd(command, attr):
    return Template(command).substitute(**attr)

def execute_block(block, attributes, indent=0, no_exit=False):
    if type(block) == dict:
        for k, v in block.items():
            execute_block(k, attributes=attributes, indent=indent)
            for b in block[k]: 
                execute_block(b, attributes=attributes, indent=indent+1)
            if not no_exit:
                print ('{}exit'.format(' '*indent*3))
    else:
        print ('{}{}'.format(' '*indent*2, _cmd(block, attributes)))

def commands(template_file, section, attr=None, no_exit=False):
    with open(template_file) as file:
        template = dotdict(yaml.load(file, Loader=yaml.FullLoader))
        base = template[section]
        for cmd in base:
            execute_block(cmd, attributes=attr, no_exit=no_exit)

In [ ]:
print ('base commands to run:')
print ('=====================')
for name, attr in topo.getLeafs():
    print ('{}:'.format(name))
    print ('-'*(len(name)+1))
    commands('base_config.yaml', 'init', attr)

    print ('')

In [ ]:
print ('Management commands to run:')
print ('===========================')
for name, attr in topo.getLeafs():
    print ('{}:'.format(name))
    print ('-'*(len(name)+1))
    options = { 'port': attr['management-port'], 'ip': attr['management-ip']}
    commands('management_config.yaml', 'interface', options)

In [ ]:
print ('Interface commands to run:')
print ('==========================')
for name, _ in topo.getLeafs():
    print ('{}:'.format(name))
    print ('-'*(len(name)+1))

    for _, adj, attr in topo.G.edges(name, data=True):
        options = {
            'port': attr['ports'][name],
            'description': adj
        }
        commands('base_config.yaml', 'interface', options)

In [ ]:
print ('BGP commands to run:')
print ('====================')
for name, attr in topo.getLeafs():
    print ('{}:'.format(name))
    print ('-'*(len(name)+1))
    _type = topo.G.nodes[name]['type']
    peer_group = {'spine': 'LEAF', 'leaf': 'SPINE'}
    option = { **{
                    'ip': '{}/32'.format(attr['router-id']),
                    'router_id': attr['router-id'],
                    'peer_group': peer_group[_type]},
                **attr }
    commands('bgp_config.yaml', 'generic', attr=option, no_exit=True)
    commands('bgp_config.yaml', 'additionnal_{}'.format(_type), attr=option)

    # get all neighbors
    for _, adj, attr in topo.G.edges(name, data=True):
        options = {
            'port': attr['ports'][name],
            'description': adj,
            'peer_group': topo.G.nodes[adj]['type'].upper()
        }
        commands('bgp_config.yaml', 'neighbor_unnumbered', attr=options)

In [ ]:
print ('VxLAN commands to run:')
print ('======================')

for name, attr in topo.getLeafs():
    print ('{}:'.format(name))
    print ('-'*(len(name)+1))

    overlay_source_address = attr.get('overlay-source-address', None)
    if not overlay_source_address:
        continue

    options = {'ip': overlay_source_address, 'mac': attr['anycast-mac-address']}


    commands('overlay_config.yaml', 'base', attr=options)
    commands('overlay_config.yaml', 'anycast', attr=options)

    vlans = attr.get('vlans', {})
    for vid, options in vlans.items():
        commands('overlay_config.yaml', 'vlan', attr={**options, **{'anycast': options['anycast-ip-address']}})
        for port in options['interfaces']:
            commands('overlay_config.yaml', 'access-port', attr={**options, **{'port':port}})
    